# Data collected from OP's website op-koti.fi. Initially I used webscraping technique to collect data. But using the API was more dynamic and easy to handle

In [75]:
import numpy as np
import pandas as pd
import requests
# import math
import re
from pandas import json_normalize

pd.options.display.max_columns = None
# display(df)

In [76]:
URL = 'https://op-koti.fi/api/apartments?mode=sale&featureGroup=apartment&orderBy=created&order=desc&offset=0&limit=4500'
# request the URL and parse the JSON
response = requests.get(URL)
response.raise_for_status() # raise exception if invalid response
data = response.json()['listings']

# normalizing the json data into a dataframe
df = json_normalize(data, max_level = 1)

In [77]:
df.head(10)

,id,kipinaId,listingType,commission,listingGroup,source,created,featureGroups,floor,year,imageUrl,showings,rooms,numberOfRooms,price,debtFreePrice,rentPerMonth,location.streetAddress,location.postalCode,location.postOffice,location.city,location.cityId,location.region,location.regionId,location.district,location.districtId,location.streetId,location.coordinates,housing.livingArea,housing.totalArea,housing.siteArea,housing.rooms,housing.numberOfRooms,pricing.price,livingArea.size,livingArea.displayUnit,totalArea.size,totalArea.displayUnit,siteArea.size,siteArea.displayUnit,pricing.debtFreePrice,siteArea,videoUrl,virtualPresentationUrl,livingArea,totalArea,newHousing.newHousingId,newHousing.name
0,510933,435642,90,K,82,Kipinä,2021-10-31T09:06:20.967Z,[apartment],0.0,2004.0,https://op-koti.fi/kuvat/510933/040a84d274fc1e...,[],"5h, k, tkh, kph, s, 2x wc, 3x varasto/harraste...",5.0,357000.00,NaN,None,Rieniläntie 25,38420,SASTAMALA,Sastamala,790,Sastamala,488,Häijää,8799.0,f8c0c9da0f2c813e9c834d2a4c6a1a409c0bcdaf,"{'latitude': 61.50025, 'longitude': 23.10317}","{'size': 214, 'displayUnit': 'm2'}","{'size': 312, 'displayUnit': 'm2'}","{'size': 5500, 'displayUnit': 'm2'}","5h, k, tkh, kph, s, 2x wc, 3x varasto/harraste...",5.0,357000.00,214.0,m2,312.0,m2,5500.0,m2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,517946,442634,91,O,82,Kipinä,2021-10-31T09:05:55Z,[apartment],1.0,1984.0,https://op-koti.fi/kuvat/517946/cb4144428a20dc...,"[{'showingType': 'unknown', 'date': '2.11.', '...","3h,k,s",3.0,96000.00,96000.0,None,Jutaajantie 19 B,96440,ROVANIEMI,Rovaniemi,698,Rovaniemi,358,Ounasrinne,5906.0,07c1d64b724495f45ac5fdc321d189892a341996,"{'latitude': 66.48165, 'longitude': 25.82997}","{'size': 78.5, 'displayUnit': 'm2'}","{'size': 78.5, 'displayUnit': 'm2'}","{'size': 3097, 'displayUnit': 'm2'}","3h,k,s",3.0,96000.00,78.5,m2,78.5,m2,3097.0,m2,96000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,518324,443012,89,O,82,Kipinä,2021-10-31T08:05:57.822Z,[apartment],3.0,1961.0,https://op-koti.fi/kuvat/518324/8f80f734aa5440...,[],"1h,kt,kph/wc,eteinen",1.0,157000.00,157000.0,None,Tammelan Puistokatu 30-32 D,33100,TAMPERE,Tampere,837,Keskusta,63,Tammela,1042.0,4292b987b5b5f97daad0ae7526e654819532306d,"{'latitude': 61.50008, 'longitude': 23.77932}","{'size': 27.5, 'displayUnit': 'm2'}","{'size': 27.5, 'displayUnit': 'm2'}","{'size': 1800, 'displayUnit': 'm2'}","1h,kt,kph/wc,eteinen",1.0,157000.00,27.5,m2,27.5,m2,1800.0,m2,157000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,510967,435676,89,O,82,Kipinä,2021-10-31T07:05:37.988Z,[apartment],3.0,1953.0,https://op-koti.fi/kuvat/510967/1b25b0c6beb056...,[],"4h,k,parv",4.0,64980.92,69000.0,None,Nahkurinkatu 10 A,94100,KEMI,Kemi,240,Kemi,167,Keskusta,13072.0,a00b9b1d52187e944307703cf1a875ef27785691,"{'latitude': 65.73387, 'longitude': 24.56492}","{'size': 100, 'displayUnit': 'm2'}","{'size': 100, 'displayUnit': 'm2'}","{'size': 2164, 'displayUnit': 'm2'}","4h,k,parv",4.0,64980.92,100.0,m2,100.0,m2,2164.0,m2,69000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,517750,442438,90,K,82,Kipinä,2021-10-30T16:05:28.617Z,[apartment],0.0,1979.0,https://op-koti.fi/kuvat/517750/fa23160150175d...,"[{'showingType': 'unknown', 'date': '1.11.', '...","5h,k,kph,s,2xvh,2wc",5.0,240000.00,NaN,None,Siilinkarinkatu 19,33410,TAMPERE,Tampere,837,Länsi,65,Lentävänniemi,1050.0,cae1badcc5c828dad5943018f3e607c8c51ea410,"{'latitude': 61.52838, 'longitude': 23.71935}","{'size': 95, 'displayUnit': 'm2'}","{'size': 122, 'displayUnit': 'm2'}","{'size': 836, 'displayUnit': 'm2'}","5h,k,kph,s,2xvh,2wc",5.0,240000.00,95.0,m2,122.0,m2,836.0,m2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,518405,443092,90,K,82,Kipinä,2021-10-30T14:05:56.702Z,[apartment],0.0,1968.0,https://op-koti.fi/kuvat/518405/d6a45c2d45bfeb...,[],"5h, k, s, at",5.0,120000.00,NaN,None,Mattilankatu 17,32200,LOIMAA,Loimaa,430,Loimaa,246,,NaN,06a76f389116d8e332f62c735fb1ac9481efa836,"{'latitude': 60.85004, 'longitude': 23.0435}","{'size': 120, 'displayUnit': 'm2'}","{'size': 163, 'displayUnit': 'm2'}","{'size': 816, 'displayUnit': 'm2'}","5

It can be seen above that the dataframe contains multiple columns that are irrelevant to us. Relevant columns are selected as shown below.

In [78]:
df.columns

Index(['id', 'kipinaId', 'listingType', 'commission', 'listingGroup', 'source',
       'created', 'featureGroups', 'floor', 'year', 'imageUrl', 'showings',
       'rooms', 'numberOfRooms', 'price', 'debtFreePrice', 'rentPerMonth',
       'location.streetAddress', 'location.postalCode', 'location.postOffice',
       'location.city', 'location.cityId', 'location.region',
       'location.regionId', 'location.district', 'location.districtId',
       'location.streetId', 'location.coordinates', 'housing.livingArea',
       'housing.totalArea', 'housing.siteArea', 'housing.rooms',
       'housing.numberOfRooms', 'pricing.price', 'livingArea.size',
       'livingArea.displayUnit', 'totalArea.size', 'totalArea.displayUnit',
       'siteArea.size', 'siteArea.displayUnit', 'pricing.debtFreePrice',
       'siteArea', 'videoUrl', 'virtualPresentationUrl', 'livingArea',
       'totalArea', 'newHousing.newHousingId', 'newHousing.name'],
      dtype='object')

In [79]:
cols = ['id', 'listingType', 'floor', 'year', 'rooms', 'numberOfRooms', 'price', 'debtFreePrice',
       'location.city', 'location.region', 'location.district','location.postalCode',
       'livingArea.size', 'totalArea.size']

df = df[cols]

Renaming the cols

In [80]:
df = df.rename(columns = {'location.city':'city','location.region':'region','location.district':'district','location.postalCode':'postalCode','livingArea.size':'livingArea','totalArea.size':'totalArea', 'year':'yearBuilt'})

Mapping the listingType column to the apporpriate type of house (extracted from the website manually)

In [81]:
lType = {89:'Kerrostalo',90:'Omakotitalo',91:'Rivitalo',92:'Paritalo',93:'Erillistalo',112:'Puutalo',113:'Luhtitalo',470:'Kytketty paritalo'}
df['listingType'] = df['listingType'].map(lType)

In [82]:
# some values in df.district are ''. So i decided to fill them with np.nan and then remove them altogether in the next step
df[df.district == ''] = np.nan

In [83]:
def count_NAs(df):
    for col in df:
        print(col," : ",df[col].isnull().sum())
        
# def homeSizeLabels(area):
#     """these ranges were calulated by using pd.cut(). 1=Smallest...4=Largest"""
#     if area in range(0,100):
#         return 1
#     elif area in range(100,175):
#         return 2
#     elif area in range(175,250):
#         return 3
#     else:
#         return 4 

In [84]:
# df = df.dropna(inplace = True)
count_NAs(df)

id  :  201
listingType  :  201
floor  :  562
yearBuilt  :  206
rooms  :  316
numberOfRooms  :  316
price  :  315
debtFreePrice  :  1267
city  :  201
region  :  201
district  :  201
postalCode  :  201
livingArea  :  316
totalArea  :  316


There are some columns with null values for debtFreePrice. Those values are replaced by price. 

In [85]:
df.loc[df.debtFreePrice.isnull() & df.price.notnull(),['debtFreePrice']] = df.price

Calculating building age from 'yearBuilt'. Some buildings are supposed to be completed in the future. So any building that gets completed in the future year are also grouped into '1 year old buildings'

In [86]:
df['buildingAge'] = [1 if x > 2021 else (pd.to_datetime('today').year - pd.to_datetime(x, format = "%Y").year) for x in df.yearBuilt]
# df['centrum'] = [1 if x == 'Keskusta' else 0 for x in df.district]
# df['howBig'] = [homeSizeLabels(x) for x in df.livingArea]

In [87]:
# hideRows = ['listingGroup', 'yearBuilt', 'imageUrl', 'streetAddress', 'postalCode', 'region', 'link']
df.drop('yearBuilt', axis = 1, inplace = True)
df.reset_index(drop = True)

,id,listingType,floor,rooms,numberOfRooms,price,debtFreePrice,city,region,district,postalCode,livingArea,totalArea,buildingAge
0,510933,Omakotitalo,0.0,"5h, k, tkh, kph, s, 2x wc, 3x varasto/harraste...",5.0,357000.00,357000.0,Sastamala,Sastamala,Häijää,38420,214.0,312.0,17.0
1,517946,Rivitalo,1.0,"3h,k,s",3.0,96000.00,96000.0,Rovaniemi,Rovaniemi,Ounasrinne,96440,78.5,78.5,37.0
2,518324,Kerrostalo,3.0,"1h,kt,kph/wc,eteinen",1.0,157000.00,157000.0,Tampere,Keskusta,Tammela,33100,27.5,27.5,60.0
3,510967,Kerrostalo,3.0,"4h,k,parv",4.0,64980.92,69000.0,Kemi,Kemi,Keskusta,94100,100.0,100.0,68.0
4,517750,Omakotitalo,0.0,"5h,k,kph,s,2xvh,2wc",5.0,240000.00,240000.0,Tampere,Länsi,Lentävänniemi,33410,95.0,122.0,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3314,kiinteisto-oy-joensuun-koskikatu-9,Kerrostalo,NaN,None,NaN,NaN,NaN,Joensuu,Joensuu,Keskusta,80100,NaN,NaN,6.0
3315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3316,604375,Kerrostalo,1.0,"1h,k",1.0,18500.00,18500.0,Virrat,Virrat,Nallela,34800,30.0,30.0,51.0
3317,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The values in the 'rooms' column are separated using , or + or . or * and need to be cleaned

In [88]:
df.dropna(inplace = True)
count_NAs(df)

id  :  0
listingType  :  0
floor  :  0
rooms  :  0
numberOfRooms  :  0
price  :  0
debtFreePrice  :  0
city  :  0
region  :  0
district  :  0
postalCode  :  0
livingArea  :  0
totalArea  :  0
buildingAge  :  0


In [89]:
def clean_rooms(item):
    delimiters = "+", ","
    # >>> example = "stackoverflow (c) is awesome... isn't it?"
    regexPattern = '|'.join(map(re.escape, delimiters))
    return([re.sub('\\+|,|\\.|\\*| ', '', x) for x in re.split(regexPattern, item)])

df.rooms = [clean_rooms(x) for x in df.rooms]

These functions take a single entry from df.rooms and detects the features. These features are then represented with new columns using 1 or 0 denoting the presence or absence of the feature

In [90]:
def detect_centrum(postalCode):
    if postalCode[2:] == '100':
        return 1
    else:
        return 0

df['centrum'] = [detect_centrum(x) for x in df.postalCode]

def detect_sauna(room): #v4
    if 's' in room or str(room).find('sauna') != -1:
        return 1
    else:
        return 0
df['hasSauna'] = [detect_sauna(x) for x in df.rooms]

def detect_balcony(room): #v3
    if 'p' in room or str(room).find('parv') != -1:
        return 1
    else:
        return 0
df['hasBalcony'] = [detect_balcony(x) for x in df.rooms]

def detect_parking(room): #v3
    if 'ak' in room or 'at' in room or str(room).find('auto') != -1:
        return 1
    else:
        return 0
df['hasParking'] = [detect_parking(x) for x in df.rooms]

def detect_walk_in_closet(room): #v3
    if 'v' in room or 'vh' in room or str(room).find('vaate') != -1:
        return 1
    else:
        return 0
df['hasWalkInCloset'] = [detect_walk_in_closet(x) for x in df.rooms]

def detect_storage_room(room):
    if str(room).find('var') != -1:
        return 1
    else:
        return 0
df['hasStorageRoom'] = [detect_storage_room(x) for x in df.rooms]

In [91]:
df.drop('rooms', axis = 1, inplace = True)

In [92]:
df['id'] = df['id'].astype(str)
df['floor'] = df['floor'].astype(int)
df['numberOfRooms'] = df['numberOfRooms'].astype(int)
df['postalCode'] = df['postalCode'].astype(str)
df['buildingAge'] = df['buildingAge'].astype(int)

Exporting the dataframe to a csv file

In [93]:
df.to_csv(f'op-koti.csv', sep=',', encoding='utf-8', index=False)